In [1]:
!pip3 install langchain langchain-google python-dotenv sqlite3 requests flask pydantic

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Samay Mehar/Downloads/dulcet-fuze-460017-k1-4c6e9a44b930.json"
from dotenv import load_dotenv
load_dotenv()
IG_ACCESS_TOKEN=os.getenv("IG_ACCESS_TOKEN")
IG_ACCOUNT_ID=os.getenv("IG_ACCOUNT_ID")
PAGE_ID        = os.getenv("PAGE_ID")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")
VERIFY_TOKEN    = os.getenv("VERIFY_TOKEN")
from flask import Flask, request
import requests
import threading
import sqlite3
import json
from datetime import datetime
from typing import Optional,Dict,Any,List
from pydantic import BaseModel,Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langgraph.graph import StateGraph,END
from langchain.schema import HumanMessage, AIMessage

In [3]:
def init_db():
    conn = sqlite3.connect('hotel_bookings.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS bookings(
        id INTEGER PRIMARY KEY,
        guest_name TEXT,
        guest_id TEXT,
        room_type TEXT,
        check_in TEXT,
        check_out TEXT,
        num_guests INTEGER,
        booking_time TEXT,
        status TEXT
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS sessions(
        user_id TEXT PRIMARY KEY,
        state_json TEXT
    )''')
    conn.commit()
    conn.close()
init_db()

In [ ]:
class Booking(BaseModel):
    guest_name: str
    guest_id: str
    room_type: str
    check_in: str
    check_out: str
    num_guests: int
    booking_time: str = Field(default_factory=lambda: datetime.now().isoformat())
    status: str = "confirmed"

class AgentState(BaseModel):
    user_id: str
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    booking_info: Optional[Dict[str, Any]] = None
    current_task: Optional[str] = None
    task_complete: bool = False

def parse_date(s: str) -> Optional[str]:
    try:
        return datetime.strptime(s, '%Y-%m-%d').date().isoformat()
    except:
        return None

In [ ]:
def load_state(user_id: str):
    conn = sqlite3.connect('hotel_bookings.db')
    c = conn.cursor()
    c.execute("SELECT state_json FROM sessions WHERE user_id=?", (user_id,))
    row = c.fetchone()
    conn.close()
    
    if row and row[0]:
        state_data = json.loads(row[0])
        state_data["user_id"] = user_id
        return AgentState.model_validate(state_data)
    return AgentState(user_id=user_id)

def save_state(user_id: str, state: AgentState):
    j = state.model_dump_json()
    conn = sqlite3.connect('hotel_bookings.db')
    c = conn.cursor()
    c.execute("REPLACE INTO sessions(user_id,state_json) VALUES(?,?)", (user_id, j))
    conn.commit()
    conn.close()

In [6]:
def get_llm():
    return ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        google_api_key=GEMINI_API_KEY,
        temperature=0.7
    )

ROOM_TYPES = {"standard": {}, "deluxe": {}, "suite": {}}

def add_booking(b: Booking) -> int:
    try:
        conn = sqlite3.connect('hotel_bookings.db')
        c = conn.cursor()
        c.execute(
            "INSERT INTO bookings(guest_name, guest_id, room_type, check_in, check_out, num_guests, booking_time, status) VALUES(?,?,?,?,?,?,?,?)",
            (b.guest_name, b.guest_id, b.room_type, b.check_in, b.check_out, 
             b.num_guests, b.booking_time, b.status)
        )
        conn.commit()
        lastid = c.lastrowid
        conn.close()
        return lastid
    except Exception as e:
        print(f"Database error: {str(e)}")
        conn.rollback()
        conn.close()
        raise e

In [ ]:
def get_booking(guest_id: str):
    conn = sqlite3.connect('hotel_bookings.db')
    cursor = conn.cursor()
    cursor.execute(
        '''SELECT id, guest_name, room_type, check_in, check_out, num_guests, status 
           FROM bookings WHERE guest_id = ? AND status != 'cancelled' ''',
        (guest_id,)
    )
    result = cursor.fetchall()
    conn.close()
    
    bookings = []
    for row in result:
        bookings.append({
            "id": row[0],
            "guest_name": row[1],
            "room_type": row[2],
            "check_in": row[3],
            "check_out": row[4],
            "num_guests": row[5],
            "status": row[6]
        })
    return bookings

def update_booking(booking_id: int, updates: Dict[str, Any]):
    conn = sqlite3.connect('hotel_bookings.db')
    cursor = conn.cursor()
    
    set_clause = ", ".join([f"{key} = ?" for key in updates.keys()])
    values = list(updates.values())
    values.append(booking_id)
    
    cursor.execute(
        f'''UPDATE bookings SET {set_clause} WHERE id = ?''',
        values
    )
    conn.commit()
    conn.close()

def cancel_booking(booking_id: int):
    update_booking(booking_id, {"status": "cancelled"})

# Agent nodes
def parse_intent(state: AgentState) -> AgentState:
    llm = get_llm()
    
    system_prompt = """You are an AI assistant for a hotel. Your job is to determine the user's intent from their message.
    Classify the intent into one of these categories:
    - book_room: User wants to book a new room
    - check_booking: User wants to check their existing booking
    - reschedule: User wants to change dates for an existing booking
    - cancel: User wants to cancel a booking
    - room_info: User wants information about room types, amenities, or prices
    - general_info: User is asking about hotel policies, services, or other general information
    - greeting: User is just saying hello or making small talk
    - other: Any other intent not covered above
    
    Respond with ONLY the intent category (e.g., "book_room") and nothing else."""
    
    messages = state.messages
    
    if not messages:
        return state
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    history = []
    for msg in messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": messages[-1].get("content", "")
    })
    
    intent = response.content.strip().lower()
    state.current_task = intent
    
    return state

def handle_booking(state: AgentState) -> dict:
    llm = get_llm()
    
    system_prompt = """You are a hotel booking assistant. Extract the booking details from the user's message.
    You need to extract:
    - guest_name: The name of the person making the booking
    - room_type: The type of room (standard, deluxe, or suite)
    - check_in: The check-in date (in YYYY-MM-DD format)
    - check_out: The check-out date (in YYYY-MM-DD format)
    - num_guests: The number of guests

    If any information is missing, respond with a JSON containing only the fields you can extract and null values for missing fields.
    Format your response as a valid JSON object with these fields and nothing else."""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    # Convert message format
    history = []
    for msg in state.messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": state.messages[-1].get("content", "")
    })
    print('LLM response:', response.content)  # Add this line for debugging
    
    try:
        json_str = response.content.strip()
        json_str = json_str.replace("```json", "").replace("```", "").strip()
        booking_info = json.loads(json_str)
        state.booking_info = booking_info
        
        # Check if we have all required information
        required_fields = ["guest_name", "room_type", "check_in", "check_out", "num_guests"]
        missing_fields = [field for field in required_fields if not booking_info.get(field)]
        
        if missing_fields:
            state.task_complete = False
            response_message = f"Missing: {', '.join(missing_fields)}. Please provide these details."
        else:
            booking_info["guest_id"] = state.user_id
            try:
                booking = Booking(**booking_info)
                booking_id = add_booking(booking)
                state.task_complete = True
                response_message = f"Success! Booking ID: {booking_id}"
            except Exception as e:
                print(f"Database error: {str(e)}")
                response_message = "Error saving booking. Please try again."
    except json.JSONDecodeError:
        response_message = "I couldn't parse your booking details. Please provide: Name, room type, dates, and guests."
    
    state.messages.append({"role": "assistant", "content": response_message})
    return state

def handle_check_booking(state: AgentState) -> dict:
    guest_id = state.user_id
    bookings = get_booking(guest_id)
    
    if bookings:
        booking_details = []
        for booking in bookings:
            details = f"Booking #{booking['id']}: {booking['room_type']} room from {booking['check_in']} to {booking['check_out']} for {booking['num_guests']} guest(s). Status: {booking['status']}"
            booking_details.append(details)
        
        response_message = "Here are your current bookings:\n" + "\n".join(booking_details)
    else:
        response_message = "You don't have any active bookings with us. Would you like to make a new reservation?"
    
    state.messages.append({"role": "assistant", "content": response_message})
    state.task_complete = True
    return state

def handle_reschedule(state: AgentState) -> dict:
    llm = get_llm()
    
    system_prompt = """You are a hotel booking assistant. Extract the rescheduling details from the user's message.
    You need to extract:
    - booking_id: The ID of the booking to reschedule (a number)
    - new_check_in: The new check-in date (in YYYY-MM-DD format)
    - new_check_out: The new check-out date (in YYYY-MM-DD format)

    If any information is missing, respond with a JSON containing only the fields you can extract and null values for missing fields.
    Format your response as a valid JSON object with these fields and nothing else."""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    history = []
    for msg in state.messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": state.messages[-1].get("content", "")
    })
    
    try:
        reschedule_info = json.loads(response.content)
        required_fields = ["booking_id", "new_check_in", "new_check_out"]
        missing_fields = [field for field in required_fields if reschedule_info.get(field) is None]
        
        if missing_fields:
            if "booking_id" in missing_fields:
                guest_id = state.user_id
                bookings = get_booking(guest_id)
                
                if not bookings:
                    response_message = "You don't have any active bookings with us. Would you like to make a new reservation?"
                    state.task_complete = True
                else:
                    booking_list = "\n".join([f"Booking #{b['id']}: {b['room_type']} room from {b['check_in']} to {b['check_out']}" for b in bookings])
                    response_message = f"Which booking would you like to reschedule? Here are your current bookings:\n{booking_list}\nPlease provide the booking ID number."
            else:
                response_message = f"I need a bit more information to reschedule your booking. Could you please provide: {', '.join(missing_fields)}?"
            
            state.task_complete = False
        else:
            updates = {
                "check_in": reschedule_info["new_check_in"],
                "check_out": reschedule_info["new_check_out"]
            }
            
            update_booking(reschedule_info["booking_id"], updates)
            
            state.task_complete = True
            response_message = f"I've rescheduled your booking #{reschedule_info['booking_id']} to the new dates: Check-in on {reschedule_info['new_check_in']} and check-out on {reschedule_info['new_check_out']}. Is there anything else you need help with?"
    except Exception as e:
        state.task_complete = False
        response_message = "I couldn't process your rescheduling request. Could you please provide the booking ID number and the new check-in and check-out dates?"
    
    state.messages.append({"role": "assistant", "content": response_message})
    return state

def handle_cancel(state: AgentState) -> dict:
    llm = get_llm()
    
    system_prompt = """You are a hotel booking assistant. Extract the cancellation details from the user's message.
    You need to extract:
    - booking_id: The ID of the booking to cancel (a number)

    Format your response as a valid JSON object with this field and nothing else."""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    # Convert message format
    history = []
    for msg in state.messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": state.messages[-1].get("content", "")
    })
    
    try:
        cancel_info = json.loads(response.content)
        
        if cancel_info.get("booking_id") is None:
            guest_id = state.user_id
            bookings = get_booking(guest_id)
            
            if not bookings:
                response_message = "You don't have any active bookings with us. Would you like to make a new reservation?"
                state.task_complete = True
            else:
                booking_list = "\n".join([f"Booking #{b['id']}: {b['room_type']} room from {b['check_in']} to {b['check_out']}" for b in bookings])
                response_message = f"Which booking would you like to cancel? Here are your current bookings:\n{booking_list}\nPlease provide the booking ID number."
                state.task_complete = False
        else:
            cancel_booking(cancel_info["booking_id"])
            
            state.task_complete = True
            response_message = f"I've cancelled your booking #{cancel_info['booking_id']}. If you change your mind, you can always make a new reservation. Is there anything else you need help with?"
    except Exception as e:
        state.task_complete = False
        response_message = "I couldn't process your cancellation request. Could you please provide the booking ID number you wish to cancel?"
    
    state.messages.append({"role": "assistant", "content": response_message})
    return state

def handle_room_info(state: AgentState) -> dict:
    llm = get_llm()
    
    system_prompt = """You are a hotel information assistant. Based on the user's query about rooms, 
    provide information about our room types, their amenities, and prices.

    We have these room types:
    - Standard Room: $100/night, accommodates 2 guests, amenities include TV, WiFi, and Air Conditioning
    - Deluxe Room: $150/night, accommodates 2 guests, amenities include TV, WiFi, Air Conditioning, Mini Bar, and City View
    - Suite: $250/night, accommodates 4 guests, amenities include TV, WiFi, Air Conditioning, Mini Bar, Living Room, Kitchenette, and Ocean View

    Provide a helpful, concise response based on what the user is asking about.
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    history = []
    for msg in state.messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": state.messages[-1].get("content", "")
    })
    
    state.messages.append({"role": "assistant", "content": response.content})
    state.task_complete = True
    return state

def handle_general_info(state: AgentState) -> dict:
    llm = get_llm()
    
    system_prompt = """You are a hotel information assistant. Based on the user's query, 
    provide information about our hotel policies, services, and facilities.

    Hotel Information:
    - Check-in time: 3:00 PM
    - Check-out time: 11:00 AM
    - Pet policy: Only service animals allowed
    - Parking: Valet parking available for $25/day
    - WiFi: Complimentary high-speed WiFi throughout the property
    - Breakfast: Continental breakfast included with all bookings
    - Facilities: Swimming pool, fitness center, spa, restaurant, and bar
    - Location: Downtown, walking distance to major attractions
    - Cancellation policy: Free cancellation up to 48 hours before check-in

    Provide a helpful, concise response based on what the user is asking about.
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    history = []
    for msg in state.messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": state.messages[-1].get("content", "")
    })
    
    state.messages.append({"role": "assistant", "content": response.content})
    state.task_complete = True
    return state

def handle_greeting(state: AgentState) -> dict:
    response_message = "Hello! Welcome to Grand Hotel. I'm your virtual assistant, how can I help you today? I can help with booking rooms, checking reservations, or answering questions about our hotel."
    state.messages.append({"role": "assistant", "content": response_message})
    state.task_complete = True
    return state

def handle_other(state: AgentState) -> dict:
    llm = get_llm()
    
    system_prompt = """You are a hotel assistant. The user has asked something that doesn't clearly fall into our standard categories.
    Provide a helpful, friendly response. If their request is hotel-related but outside your capabilities, politely explain what you can help with.
    If they're asking about something completely unrelated to the hotel, gently steer the conversation back to hotel services.
    
    You can help with:
    - Booking rooms
    - Checking reservations
    - Changing/cancelling bookings
    - Information about rooms, amenities, and hotel services
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    history = []
    for msg in state.messages[:-1]:
        if msg.get("role") == "user":
            history.append(HumanMessage(content=msg.get("content", "")))
        else:
            history.append(AIMessage(content=msg.get("content", "")))
    
    chain = prompt | llm
    response = chain.invoke({
        "history": history,
        "input": state.messages[-1].get("content", "")
    })
    
    state.messages.append({"role": "assistant", "content": response.content})
    state.task_complete = True
    return state

def router(state: AgentState) -> str:
    if state.task_complete:
        return END
    
    task = state.current_task
    if task == "book_room":
        return "handle_booking"
    elif task == "check_booking":
        return "handle_check_booking"
    elif task == "reschedule":
        return "handle_reschedule"
    elif task == "cancel":
        return "handle_cancel"
    elif task == "room_info":
        return "handle_room_info"
    elif task == "general_info":
        return "handle_general_info"
    elif task == "greeting":
        return "handle_greeting"
    else:
        return "handle_other"

def build_graph():
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("parse_intent", parse_intent)
    workflow.add_node("handle_booking", handle_booking)
    workflow.add_node("handle_check_booking", handle_check_booking)
    workflow.add_node("handle_reschedule", handle_reschedule)
    workflow.add_node("handle_cancel", handle_cancel)
    workflow.add_node("handle_room_info", handle_room_info)
    workflow.add_node("handle_general_info", handle_general_info)
    workflow.add_node("handle_greeting", handle_greeting)
    workflow.add_node("handle_other", handle_other)

    workflow.set_entry_point("parse_intent")
    workflow.add_conditional_edges(
        "parse_intent",
        router,
        {
            "handle_booking": "handle_booking",
            "handle_check_booking": "handle_check_booking",
            "handle_reschedule": "handle_reschedule",
            "handle_cancel": "handle_cancel",
            "handle_room_info": "handle_room_info",
            "handle_general_info": "handle_general_info",
            "handle_greeting": "handle_greeting",
            "handle_other": "handle_other",
            END: END
        }
    )

    for node in ["handle_booking", "handle_check_booking", "handle_reschedule", 
                "handle_cancel", "handle_room_info", "handle_general_info", 
                "handle_greeting", "handle_other"]:
        workflow.add_conditional_edges(
            node,
            lambda state: "parse_intent" if state.task_complete else END,
            {
                "parse_intent": "parse_intent",
                END: END
            }
        )
    
    return workflow.compile()

In [ ]:
class HotelBookingAgent:
    def __init__(self, user_id: str):
        init_db()
        self.graph = build_graph()
        self.state = load_state(user_id)
        self.state.user_id = user_id
        self.user_id = user_id

    def process_message(self, message: str) -> str:
        self.state.messages.append({"role": "user", "content": message})
        self.state.task_complete = False
        result = self.graph.invoke(self.state)
        self.state = AgentState(**result)
        save_state(self.user_id, self.state)
        return self.state.messages[-1]["content"] if self.state.messages else "Sorry, couldn't process."

In [9]:
app = Flask(__name__)

@app.route('/', methods=['GET'])
def health_check():
    return "🚀 Hotel Agent is running!", 200

@app.route('/webhook', methods=['GET', 'POST'], endpoint='webhook')
def handle_webhook():
    if request.method == 'GET':
        token = request.args.get('hub.verify_token')
        challenge = request.args.get('hub.challenge')
        return challenge if token == VERIFY_TOKEN else ('Forbidden', 403)

    data = request.get_json(force=True)
    for entry in data.get('entry', []):
        for msg in entry.get('messaging', []):
            uid = msg['sender']['id']
            text = msg['message']['text']
            agent = HotelBookingAgent(uid)
            resp = agent.process_message(text)
            post_url = f"https://graph.facebook.com/v15.0/{PAGE_ID}/messages?access_token={IG_ACCESS_TOKEN}"
            requests.post(post_url, json={
                "recipient": {"id": uid},
                "message":   {"text": resp}
            })
    return 'OK', 200

In [ ]:
def run_app():
    app.run(host='0.0.0.0', port=int(os.getenv("PORT", 5000)), use_reloader=False)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.99:5000
Press CTRL+C to quit


In [ ]:
test_messages = [
    "Hi, I'd like to book a room",
    "I want a deluxe room from 2025-06-14 to 2025-06-20 for 3 people. My name is Samay Mehar.",
    "Can you tell me my current bookings?",
    "I'd like to change my reservation dates",
    "Booking #1, I want to change to 2025-06-12 to 2025-06-17",
    "What types of rooms do you have?",
    "What time is check-in?",
    "I need to cancel my booking",
    "Cancel booking #1"
]

def print_bookings(user_id):
    conn = sqlite3.connect('hotel_bookings.db')
    c = conn.cursor()
    c.execute("""
        SELECT id, guest_name, room_type, check_in, check_out, num_guests, status
        FROM bookings
        WHERE guest_id = ?
    """, (user_id,))
    rows = c.fetchall()
    conn.close()
    if not rows:
        print("  (no bookings)")
    else:
        for row in rows:
            print(f"  Booking #{row[0]}: {row[1]}, {row[2]} from {row[3]} to {row[4]} for {row[5]} guests (status: {row[6]})")

user_id = "test_user1"
agent = HotelBookingAgent(user_id)

for msg in test_messages:
    print(f"\nUser → {msg}")
    bot_reply = agent.process_message(msg)
    print(f"Agent → {bot_reply}")

print("\nDatabase now contains:")
print_bookings(user_id)


User → Hi, I'd like to book a room
LLM response: ```json
{
  "guest_name": null,
  "room_type": null,
  "check_in": null,
  "check_out": null,
  "num_guests": null
}
```
Agent → Missing: guest_name, room_type, check_in, check_out, num_guests. Please provide these details.

User → I want a deluxe room from 2025-06-14 to 2025-06-20 for 3 people. My name is Samay Mehar.
LLM response: ```json
{
  "guest_name": "Samay Mehar",
  "room_type": "deluxe",
  "check_in": "2025-06-14",
  "check_out": "2025-06-20",
  "num_guests": 3
}
```
Agent → Success! Booking ID: 2

User → Can you tell me my current bookings?
Agent → Here are your current bookings:
Booking #2: deluxe room from 2025-06-14 to 2025-06-20 for 3 guest(s). Status: confirmed

User → I'd like to change my reservation dates
Agent → Here are your current bookings:
Booking #2: deluxe room from 2025-06-14 to 2025-06-20 for 3 guest(s). Status: confirmed

User → Booking #1, I want to change to 2025-06-12 to 2025-06-17
Agent → You do not have